### Unet 

- pre-trained with Car dataset: https://github.com/milesial/Pytorch-UNet/blob/master/README.md


- pre-trained with brain dataset: https://github.com/mateuszbuda/brain-segmentation-pytorch/blob/master/README.md


- Docker: a platform that allows you to develop, ship, and run applications in containers. Containers are lightweight, portable, and self-sufficient 
          environments that include everything needed to run an application—code, runtime, libraries, dependencies, and configurations.


### Unet's encoder + decoder structure VS autoencoder:

Is U-Net an Autoencoder?:
✅ Yes, it follows an encoder-decoder structure like an autoencoder.
❌ No, because it is designed for segmentation, not just reconstruction.

- UNet use Skip connections, and Cross-entropy, Dice loss, IoU loss.

Merging the segmenting results of each 3-band subset
-	Majority Voting: Assign each pixel the most frequently predicted class from different band combinations.
-	Weighted Averaging: Give more importance to specific bands or results with higher confidence scores.
-	Feature Fusion via CNN or MLP: Use a post-processing model to learn the optimal way to fuse predictions from different band combinations.



In [3]:
import torch

In [ ]:
# Check if MPS is available
print(torch.__version__)
print(torch.backends.mps.is_available())
print(torch.backends.mps.is_built())

device = torch.device("mps")
x = torch.randn((10, 10), device=device)
y = torch.randn((10, 10), device=device)
z = x + y  # this should be fine

2.5.1
True
True


In [ ]:
# Ensure using MPS (Metal) on Mac
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

# Load the model with map_location: model.pth is the saved checkpoint
# model = torch.load("model.pth", map_location=device)
# Move the model to the correct device
# model.to(device)

net = torch.hub.load('milesial/Pytorch-UNet', 'unet_carvana', pretrained=True, scale=0.5, map_location=device)
# Move the model to the correct device
net.to(device)


### From Official UNet Colab

In [1]:
import torch
model = torch.hub.load('mateuszbuda/brain-segmentation-pytorch', 'unet', in_channels=3, out_channels=1, init_features=32, pretrained=True)

/opt/anaconda3/envs/hsi/lib/python3.10/site-packages/torch/hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/mateuszbuda/brain-segmentation-pytorch/zipball/master" to /Users/seaqueue/.cache/torch/hub/master.zip
Downloading: "https://github.com/mateuszbuda/brain-segmentation-pytorch/releases/download/v1.0/unet-e012d006.pt" to /Users/seaqueue/.cache/torch/hub/checkpoints/unet-e012d006.pt


In [ ]:
# Download an example image
import urllib
url, filename = ("https://github.com/mateuszbuda/brain-segmentation-pytorch/raw/master/assets/TCGA_CS_4944.png", "TCGA_CS_4944.png")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

In [2]:
import numpy as np
from PIL import Image
from torchvision import transforms
filename = '/Users/seaqueue/Documents/AI/Research/blueberry/understand_HSI/dct_compressed_034.png'
input_image = Image.open(filename)
m, s = np.mean(input_image, axis=(0, 1)), np.std(input_image, axis=(0, 1))
preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=m, std=s),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0)

if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model = model.to('cuda')

with torch.no_grad():
    output = model(input_batch)

print(torch.round(output[0]))

/opt/anaconda3/envs/hsi/lib/python3.10/site-packages/torchvision/io/image.py:14: UserWarning: Failed to load image Python extension: 'dlopen(/opt/anaconda3/envs/hsi/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.9.dylib
  Referenced from: <FF5E77A4-1F04-398B-B781-976783A281B8> /opt/anaconda3/envs/hsi/lib/python3.10/site-packages/torchvision/image.so
  Reason: tried: '/opt/anaconda3/envs/hsi/lib/python3.10/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/opt/anaconda3/envs/hsi/lib/python3.10/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/opt/anaconda3/envs/hsi/lib/python3.10/lib-dynload/../../libjpeg.9.dylib' (no such file), '/opt/anaconda3/envs/hsi/bin/../lib/libjpeg.9.dylib' (no such file)'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])


In [3]:
# Convert to PIL image
to_pil = transforms.ToPILImage()
pil_image = to_pil(output[0])

# Save the image
pil_image.save("pca_unet_out.png")

print("Image saved")

Image saved
